In [2]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-xl')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-xl')

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.14s/it]


['yes']
['<pad> yes</s>']
tensor([[0.0290, 0.0054, 0.1668, 0.7988]])


In [5]:

# get ids in vocab
tokenizer(['yes', 'no', 'Yes', 'No', 'A', 'B', 'C', 'D'])['input_ids']
# [[0, 10932, 2], [0, 2362, 2], [0, 9904, 2], [0, 3084, 2], [0, 250, 2], [0, 387, 2], [0, 347, 2], [0, 495, 2]]
constrained_idx1 = [10932, 2362]
constrained_idx2 = [9904, 3084]

# get the scores of certain tokens
inputs = tokenizer('Is the basketball bigger then the pingpong ball ?', return_tensors="pt")
outputs = model.generate(inputs["input_ids"],
                         num_beams=1, 
                         return_dict_in_generate=True,
                         output_scores=True)
print(tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True))
print(tokenizer.batch_decode(outputs.sequences))
logits = outputs.scores[1] # skip the bos token
constrained_logits = logits[:, constrained_idx1]
print(torch.softmax(constrained_logits, dim=-1)) # 0.9015, 0.0985
constrained_logits = logits[:, constrained_idx2]
print(torch.softmax(constrained_logits, dim=-1)) # 0.0073, 0.9927

['yes']
['<pad> yes</s>']
tensor([[0.8654, 0.1346]])
tensor([[0.1760, 0.8240]])
